# Create Response Generator Dataset
This notebook creates the dataset for re-training the response generator model.

## Pre-requisites
A CSV file must be created that contains fields including input, search results, and output.

In [1]:
from citation_fetcher import Citation_Fetcher as cf
from datetime import datetime as dt
from query_parser import QueryParser as QP
import const
import numpy as np
import pandas as pd

In [2]:
RESPONSE_GEN_DS_PATH: str =  const.DATASETS_FOLDER + "response_gen_ground_truth.csv"
SQUAD_DS_PATH: str = const.DATASETS_FOLDER + "squad_ds_keyword_ground_truth.csv"
ULTRACHAT_LITE_PATH: str = const.DATASETS_FOLDER + "ultrachat_50k.csv"
REDDIT_DS_PATH: str = const.DATASETS_FOLDER + "reddit_ds.csv"

In [3]:
squad_ds: pd.DataFrame = pd.read_csv(SQUAD_DS_PATH)
squad_len = len(squad_ds)

/tmp/ipykernel_549111/1211425012.py:1: DtypeWarning: Columns (17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  squad_ds: pd.DataFrame = pd.read_csv(SQUAD_DS_PATH)


In [4]:
ultrachat_ds: pd.DataFrame = pd.read_csv(ULTRACHAT_LITE_PATH)
ultrachat_len = len(squad_ds)

In [5]:
reddit_ds: pd.DataFrame = pd.read_csv(REDDIT_DS_PATH)

In [6]:
rg_ds = squad_ds['question']

In [7]:
rg_ds = pd.DataFrame(data={'input': rg_ds, 'query': None, 'search_result': None, 'output': None})

In [8]:
rg_ds.head()

,input,query,search_result,output
0,When did Beyonce start becoming popular?,None,None,None
1,What areas did Beyonce compete in when she was...,None,None,None
2,When did Beyonce leave Destiny's Child and bec...,None,None,None
3,In what city and state did Beyonce grow up?,None,None,None
4,In which decade did Beyonce become famous?,None,None,None


In [9]:
ultrachat_ds.head()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,question,data,is_searchable
0,0,0,0,0,How do engineers design and test roller coaste...,['How do engineers design and test roller coas...,True
1,1,1,1,1,Can you discuss how religious motifs have been...,['Can you discuss how religious motifs have be...,True
2,2,2,2,2,How has the depiction of war in movies changed...,['How has the depiction of war in movies chang...,True
3,3,3,3,3,Analyze the use of mise-en-scène in Blade Runn...,"[""Analyze the use of mise-en-scène in Blade Ru...",True
4,4,4,4,4,What steps does Procter & Gamble take to promo...,['What steps does Procter & Gamble take to pro...,True


In [10]:
ultrachat_ds = ultrachat_ds.rename(columns={'question': 'input'})

In [11]:
rg_ds  = pd.concat([rg_ds, ultrachat_ds['input']], ignore_index=True)
rg_ds = rg_ds.sample(frac=1)
rg_ds = rg_ds.reset_index()
rg_ds = rg_ds[0:2000]

In [12]:
i = 0
qp = QP(const.MODELS_FOLDER + "aletheianomous_ai-keyword_extractor-v0.3.1/")
start_time = dt.now()
elapsed_time = None
total_batch = len(rg_ds)
for j in range(len(rg_ds)):
    if elapsed_time is None:
        print("Populating Search Result ", (int((i/len(rg_ds))*100)),"% (Sample ", (i+1), "/", len(rg_ds) , ")", 
                end="                   \r")
    else:
        avg_time = elapsed_time / i
        time_remaining = avg_time * (total_batch-i)
        print("Populating Search Result ", (int((i/len(rg_ds))*100)),"% (Sample ", (i+1), "/", len(rg_ds) , "Elapsed time: ",
              elapsed_time, ", Time Remaining: ", time_remaining, ")",end="                                  \r")
    user_input = rg_ds.loc[j, "input"]
    query = qp.generate_query(str(user_input))
    rg_ds.loc[i, "query"] = query
    json_out, result = cf.search_online(query)
    rg_ds.loc[i, "search_result"] = result
    i+=1
    end_time = dt.now()
    elapsed_time = end_time - start_time
    

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/sp15-chatbot/Documents/experimental-models/.env/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/sp15-chatbot/Documents/experimental-models/.env/lib/python3.10/site-p

In [13]:
reddit_ds.head()

,Unnamed: 0,0,1,2
0,0,What kind of phone(s) do you guys have?,I have a pixel. It's pretty great. Much better...,Does it really charge all the way in 15 min?
1,1,I have a pixel. It's pretty great. Much better...,Does it really charge all the way in 15 min?,"Pretty fast. I've never timed it, but it's und..."
2,2,Does it really charge all the way in 15 min?,"Pretty fast. I've never timed it, but it's und...","cool. I've been thinking of getting one, my ph..."
3,3,What kind of phone(s) do you guys have?,Samsung Galaxy J1. It's my first cell phone an...,What do you think of it? Anything you don't like?
4,4,Samsung Galaxy J1. It's my first cell phone an...,What do you think of it? Anything you don't like?,I love it. I can't think of anything I don't l...


In [14]:
reddit_ds = reddit_ds.rename(columns={"0": "input"})

In [16]:
reddit_ds.head()

,Unnamed: 0,input,1,2
0,0,What kind of phone(s) do you guys have?,I have a pixel. It's pretty great. Much better...,Does it really charge all the way in 15 min?
1,1,I have a pixel. It's pretty great. Much better...,Does it really charge all the way in 15 min?,"Pretty fast. I've never timed it, but it's und..."
2,2,Does it really charge all the way in 15 min?,"Pretty fast. I've never timed it, but it's und...","cool. I've been thinking of getting one, my ph..."
3,3,What kind of phone(s) do you guys have?,Samsung Galaxy J1. It's my first cell phone an...,What do you think of it? Anything you don't like?
4,4,Samsung Galaxy J1. It's my first cell phone an...,What do you think of it? Anything you don't like?,I love it. I can't think of anything I don't l...


In [17]:
rg_ds.head()

,index,input,query,search_result,output,0
0,112237,What type of soldiers came from Hong Kong?,type of soldiers came from Hong Kong,[1] https://asiatimes.com/2017/05/gurkhas-hist...,None,NaN
1,16187,"A user interface, UI backend, layout engine an...",user interface UI backend layout engine and ot...,[1] https://uxdesign.cc/fundamentals-of-layout...,None,NaN
2,23391,What territory did Yugoslavia leadership want ...,territory did Yugoslavia leadership want to in...,[1] https://en.wikipedia.org/wiki/Yugoslav_irr...,None,NaN
3,177298,NaN,nan,[1] https://en.wikipedia.org/wiki/NaN : In com...,NaN,How is the music industry adapting to the rise...
4,45093,What state has the highest rate of death sente...,state has the highest rate of death sentences ...,[1] https://deathpenaltyinfo.org/stories/state...,None,NaN


In [18]:
rg_ds = pd.concat([rg_ds, reddit_ds['input']], ignore_index=True)

In [19]:
rg_ds.to_csv(RESPONSE_GEN_DS_PATH)